# IMPORTS

In [1]:
import os, sys
import pandas as pd
import numpy as np
import sqlite3
from scripts.log_processing.plotting import *
import ntpath
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display
from scripts.log_processing.plotting import *
import re

%matplotlib widget

# SET EXPERIMENT AND LOAD DATABASE

In [2]:
folder = 'experiments/BICY2020_modified/logs/experiment11-article2/'
figure_folder = folder + 'figures/'
make_folder(figure_folder)

In [7]:
sample_rate = 5
configs = load_config_file(folder)
configs['pc_files'] = configs.pc_files.map(os.path.normpath )
configs['pcs'] = configs.pc_files.map(lambda v : ntpath.basename(v)[0:-4])
min_layers = ['u04_40', 'u08_21', 'u12_14', 'u16_11', 'u20_09', 'u24_07', 'u28_06', 'u32_06', 'u36_06', 'u40_05', 'u44_05', 'u48_04', 'u52_04', 'u56_04']
db = sqlite3.connect(folder + 'experiment_results.sqlite')
print('COLUMNS', configs.columns.values)
# print(configs)

COLUMNS ['experiment' 'group' 'mazeFile' 'numStartingPositions' 'pc_files'
 'traces' 'numEpisodes' 'pcs']


# Experiment 11

### Locally uniform experiment

In [4]:
mazes = ['M0', 'M1']
traces = [0 ,0.7]
m = mazes[0]
t = traces[1]

locally_uniform_folder = os.path.join(figure_folder,'locally_uniform/')
make_folder(locally_uniform_folder)


In [5]:
maze_configs = configs[configs.mazeFile.str.contains(m + '.xml')]
maze_configs = maze_configs[maze_configs.traces == t].copy()
maze_configs['pcs'] = maze_configs.pc_files.map(lambda v : ntpath.basename(v)[0:-4])
print(f'{len(maze_configs)}/{len(configs)}')

min_layer_configs = maze_configs[maze_configs.pc_files.str.contains('|'.join(min_layers[4:]))]
print(f'{len(min_layer_configs)}/{len(maze_configs)}')

locally_uniform_layers = maze_configs[maze_configs.pc_files.str.contains( 'lu')]
print(f'{len(locally_uniform_layers)}/{len(maze_configs)}')

experiment_configs = pd.concat([min_layer_configs, locally_uniform_layers])
print(f'{len(experiment_configs)}/{len(maze_configs)}')
        

experiment_configs['base layer'] = experiment_configs.pcs.map(lambda l : re.findall('\d+', l)[-2])
experiment_configs['legend groups'] = experiment_configs.pcs.map(lambda l : 'original' if l[0]=='u' else 'goal' if l[2] == '0' else 'goal and gap')

print(experiment_configs[['mazeFile','pcs','traces','base layer','legend groups']])


115/13290
10/115
10/115
20/115
       mazeFile        pcs  traces base layer legend groups
config                                                     
c41      M0.xml     u20_09     0.7         20      original
c57      M0.xml     u24_07     0.7         24      original
c73      M0.xml     u28_06     0.7         28      original
c89      M0.xml     u32_06     0.7         32      original
c105     M0.xml     u36_06     0.7         36      original
c121     M0.xml     u40_05     0.7         40      original
c139     M0.xml     u44_05     0.7         44      original
c155     M0.xml     u48_04     0.7         48      original
c173     M0.xml     u52_04     0.7         52      original
c191     M0.xml     u56_04     0.7         56      original
c13105   M0.xml  lu0_20_16     0.7         20          goal
c13107   M0.xml  lu0_24_16     0.7         24          goal
c13109   M0.xml  lu0_28_16     0.7         28          goal
c13111   M0.xml  lu0_32_16     0.7         32          goal
c13113   

In [6]:
location = -1
num_episodes = configs['numEpisodes'].max() / experiment_configs['numStartingPositions'].max()
last_episode = -sample_rate % num_episodes
indices = [np.uint16(c[1:]) for c in experiment_configs.index]  # config numbers

categories = ['original', 'goal', 'goal and gap']
if m == 'M0':
    categories = categories[0:-1]

summaries = load_summaries(db, indices, location)
summaries['base layer'] = summaries.config.map(lambda c : experiment_configs.loc['c'+str(c),'base layer'])
summaries['legend groups'] = summaries.config.map(lambda c : experiment_configs.loc['c'+str(c),'legend groups'])
fill_cat = pd.Categorical(summaries['legend groups'], categories= categories)
summaries = summaries.assign(fill_cat = fill_cat)

runtimes_last_episode = load_episode_runtimes(db, indices, location, last_episode)
runtimes_last_episode['base layer'] = runtimes_last_episode.config.map(lambda c : experiment_configs.loc['c'+str(c),'base layer'])
runtimes_last_episode['legend groups'] = runtimes_last_episode.config.map(lambda c : experiment_configs.loc['c'+str(c),'legend groups'])
fill_cat = pd.Categorical(runtimes_last_episode['legend groups'], categories= categories)
runtimes_last_episode = runtimes_last_episode.assign(fill_cat = fill_cat)





print(summaries)
print(runtimes_last_episode)

       config  location  episode      steps base layer legend groups  fill_cat
0          41       255        0  15.985178         20      original  original
1          41       255        5   4.175132         20      original  original
2          41       255       10   2.414961         20      original  original
3          41       255       15   1.685946         20      original  original
4          41       255       20   1.262936         20      original  original
...       ...       ...      ...        ...        ...           ...       ...
39995   13123       255     9975   0.054213         56          goal      goal
39996   13123       255     9980   0.058145         56          goal      goal
39997   13123       255     9985   0.055310         56          goal      goal
39998   13123       255     9990   0.058045         56          goal      goal
39999   13123       255     9995   0.057477         56          goal      goal

[40000 rows x 7 columns]
      config  location  ep

In [7]:
group_name = f'{m}'
plot_title = f'Maze {m[1:]} - Trace {t}'

# save_folder = os.path.join(locally_uniform_folder , group_name, '')
# make_folder(save_folder)

def plot_box_plot(data, y_column, x_title, y_title, plot_title):
    print(data)
    p0 = ggplot(data, aes('base layer', y_column, fill="fill_cat" )) \
         + geom_jitter(alpha=0.3, position=position_jitterdodge(dodge_width=0.75), mapping=aes(group="fill_cat")) \
         + geom_boxplot(alpha=0.7, notch=False, outlier_alpha=0) \
         + labs(x=x_title, y=y_title, title=plot_title) \
         + theme(axis_text_x=element_text(rotation=45, hjust=0.5)) \
         + scale_fill_brewer(type="qual", palette=1, name="Group") \
         + coord_cartesian(ylim=[0,1])              
    return p0

legend_values = runtimes_last_episode.config.unique()
# print(legend_values)

# %matplotlib inline
plot_box_plot(runtimes_last_episode, 'steps',
                      'Base Layer', 'Extra Steps', plot_title)

      config  location  episode     steps base layer legend groups  fill_cat
0         41       255     9995  0.029980         20      original  original
1         41       255     9995  0.047133         20      original  original
2         41       255     9995  0.026652         20      original  original
3         41       255     9995  0.000000         20      original  original
4         41       255     9995  0.023925         20      original  original
...      ...       ...      ...       ...        ...           ...       ...
1995   13123       255     9995  0.063637         56          goal      goal
1996   13123       255     9995  0.121710         56          goal      goal
1997   13123       255     9995  0.054804         56          goal      goal
1998   13123       255     9995  0.086469         56          goal      goal
1999   13123       255     9995  0.020907         56          goal      goal

[2000 rows x 7 columns]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ggplot: (145764936183)>

In [8]:
base_layer = '20'
group_name = f'{m}'
plot_title = f'Maze {m[1:]} - Trace {t} - Base layer {base_layer}'

plot_data = summaries[summaries['base layer']==base_layer]

def plot_time_series(data, x_column, y_column, x_title, y_title, legend_title, plot_title, xlim, ylim):
#     print(data)
    p0 = ggplot(data, aes(x_column, y_column, color='fill_cat')) \
         + geom_line() \
         + labs(x=x_title, y=y_title, title=plot_title, caption='smt') \
         + coord_cartesian(ylim=ylim, xlim=xlim) \
         + scale_color_brewer(type="qual", palette=1, name=legend_title) 

    return p0


# %matplotlib inline
plot_time_series(plot_data, 'episode', 'steps' , 'Episode', 'Extra Steps', 'Group', plot_title, [0, 500], [0, 1] )


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ggplot: (145764507149)>

In [9]:
plot_time_series(plot_data, 'episode', 'steps' , 'Episode', 'Extra Steps', 'Group2', plot_title, [0, 1000], [0, 50] )


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ggplot: (145764690917)>

### Non uniform experiment

In [98]:
# CHOOSE MAZE AND TRACE
m = 'M1'
t = 0
git_root = './'
layer_metrics_file = os.path.join(git_root, 'experiments/pc_layers/layer_metrics.csv')

# CREATE FOLDERS FOR LU EXPERIMENT
folder_nu = os.path.join(figure_folder,'non_uniform/')
folder_nu_runtimes = os.path.join(folder_nu, 'runtimes/')
make_folder(folder_nu)
make_folder(folder_nu_runtimes)

# GET LAYER METRICS 
layer_metrics = pd.read_csv(layer_metrics_file)
layer_metrics['layer'] = layer_metrics.layer.map(lambda l : os.path.normpath(os.path.join('experiments/pc_layers', l)) ) # normalize path and prepend location
layer_metrics


,layer,number of cells,total area,mean area,mean radius,mean distance,normalized mean distance
0,experiments\pc_layers\locally_uniform\lu0_20_1...,126,15.426477,0.122432,0.197143,1.502259,7.620153
1,experiments\pc_layers\locally_uniform\lu0_24_1...,79,13.390725,0.169503,0.230886,1.537851,6.660649
2,experiments\pc_layers\locally_uniform\lu0_28_1...,63,14.024070,0.222604,0.262857,1.556019,5.919638
3,experiments\pc_layers\locally_uniform\lu0_32_1...,63,18.095574,0.287231,0.297143,1.556019,5.236603
4,experiments\pc_layers\locally_uniform\lu0_36_1...,63,22.709945,0.360475,0.331429,1.556019,4.694885
...,...,...,...,...,...,...,...
123,experiments\pc_layers\uniform\u56_20.csv,560,551.713935,0.985203,0.560000,1.424010,2.542874
124,experiments\pc_layers\uniform\u56_25.csv,875,862.053024,0.985203,0.560000,1.411379,2.520319
125,experiments\pc_layers\uniform\u56_30.csv,1230,1211.800251,0.985203,0.560000,1.403663,2.506540
126,experiments\pc_layers\uniform\u56_35.csv,1680,1655.141806,0.985203,0.560000,1.397716,2.495921


In [99]:
# GET MAZE CONFIGS RELEVANT TO EXPERIMENT
maze_configs = configs[configs.mazeFile.str.contains(m + '.xml')]
maze_configs = maze_configs[maze_configs.pc_files.str.contains('|'.join(['non_uniform'] + min_layers))].copy()
maze_configs

,experiment,group,mazeFile,numStartingPositions,pc_files,traces,numEpisodes,pcs
config,,,,,,,,
c208,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u04_40.csv,0.0,70000,u04_40
c209,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u04_40.csv,0.7,70000,u04_40
c210,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u08_21.csv,0.0,70000,u08_21
c211,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u08_21.csv,0.7,70000,u08_21
c220,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u12_14.csv,0.0,70000,u12_14
c221,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u12_14.csv,0.7,70000,u12_14
c234,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u16_11.csv,0.0,70000,u16_11
c235,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u16_11.csv,0.7,70000,u16_11
c248,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u20_09.csv,0.0,70000,u20_09


In [100]:
# MERGE CONFIGS AND LAYER METRICS
experiment_configs = pd.merge(maze_configs.reset_index(), layer_metrics, left_on='pc_files', right_on='layer', how='left').set_index('config')
alias = lambda l : l[0:3] if l[0] == 'u' else 'nu'
cells = lambda r : r['number of cells']
experiment_configs['layer_alias'] = experiment_configs.apply( lambda r : f'{alias(r["pcs"])} ({cells(r)})' , axis = 1 )
experiment_configs
# maze_configs.join(metrics, on='pc_files', )

,experiment,group,mazeFile,numStartingPositions,pc_files,traces,numEpisodes,pcs,layer,number of cells,total area,mean area,mean radius,mean distance,normalized mean distance,layer_alias
config,,,,,,,,,,,,,,,,
c208,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u04_40.csv,0.0,70000,u04_40,experiments\pc_layers\uniform\u04_40.csv,2200,11.058406,0.005027,0.040000,1.393318,34.832955,u04 (2200)
c209,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u04_40.csv,0.7,70000,u04_40,experiments\pc_layers\uniform\u04_40.csv,2200,11.058406,0.005027,0.040000,1.393318,34.832955,u04 (2200)
c210,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u08_21.csv,0.0,70000,u08_21,experiments\pc_layers\uniform\u08_21.csv,609,12.244672,0.020106,0.080000,1.421412,17.767654,u08 (609)
c211,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u08_21.csv,0.7,70000,u08_21,experiments\pc_layers\uniform\u08_21.csv,609,12.244672,0.020106,0.080000,1.421412,17.767654,u08 (609)
c220,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u12_14.csv,0.0,70000,u12_14,experiments\pc_layers\uniform\u12_14.csv,280,12.666902,0.045239,0.120000,1.451175,12.093122,u12 (280)
c221,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u12_14.csv,0.7,70000,u12_14,experiments\pc_layers\uniform\u12_14.csv,280,12.666902,0.045239,0.120000,1.451175,12.093122,u12 (280)
c234,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u16_11.csv,0.0,70000,u16_11,experiments\pc_layers\uniform\u16_11.csv,165,13.270087,0.080425,0.160000,1.480940,9.255873,u16 (165)
c235,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u16_11.csv,0.7,70000,u16_11,experiments\pc_layers\uniform\u16_11.csv,165,13.270087,0.080425,0.160000,1.480940,9.255873,u16 (165)
c248,experiments/setups/experiment_1.xml,g1,M1.xml,7,experiments\pc_layers\uniform\u20_09.csv,0.0,70000,u20_09,experiments\pc_layers\uniform\u20_09.csv,117,14.702654,0.125664,0.200000,1.505912,7.529561,u20 (117)


In [106]:
# GET TRACE CONFIGS AND DATA FROM LAST EPISODE
trace_configs = experiment_configs[experiment_configs.traces == t]
indices = [np.uint16(c[1:]) for c in trace_configs.index]  # config numbers

# DEFINE ORDER OF GROUPS
groups = list(trace_configs.layer_alias.values)
position = 5
categories = groups[0:position] + [groups[-1]] + groups[position:-1]

# GET DATA FROM LAST EPISODE:
location = -1
num_episodes = trace_configs['numEpisodes'].max() / trace_configs['numStartingPositions'].max()
last_episode = -sample_rate % num_episodes
runtimes_last_episode = load_episode_runtimes(db, indices, location, last_episode)
runtimes_last_episode['layer_alias'] = runtimes_last_episode.config.map(lambda c : trace_configs.loc[f'c{c}','layer_alias'])                
runtimes_last_episode['categories'] = pd.Categorical(runtimes_last_episode.layer_alias, categories= categories)

# GET RUNTIMES
erros_vs_episode_df = load_summaries(db, indices, location)
erros_vs_episode_df['layer_alias'] = erros_vs_episode_df.config.map(lambda c : trace_configs.loc[f'c{c}','layer_alias'])
erros_vs_episode_df['categories'] = pd.Categorical(erros_vs_episode_df.layer_alias, categories=categories)

runtimes_last_episode

,config,location,episode,steps,layer_alias,categories
0,208,255,9995,0.071194,u04 (2200),u04 (2200)
1,208,255,9995,0.098837,u04 (2200),u04 (2200)
2,208,255,9995,0.080196,u04 (2200),u04 (2200)
3,208,255,9995,0.049742,u04 (2200),u04 (2200)
4,208,255,9995,0.089424,u04 (2200),u04 (2200)
...,...,...,...,...,...,...
1495,13166,255,9995,0.074323,nu (79),nu (79)
1496,13166,255,9995,0.082628,nu (79),nu (79)
1497,13166,255,9995,0.084176,nu (79),nu (79)
1498,13166,255,9995,0.080696,nu (79),nu (79)


In [107]:
erros_vs_episode_df

,config,location,episode,steps,layer_alias,categories
0,208,255,0,35.856911,u04 (2200),u04 (2200)
1,208,255,5,34.858658,u04 (2200),u04 (2200)
2,208,255,10,30.503510,u04 (2200),u04 (2200)
3,208,255,15,28.009098,u04 (2200),u04 (2200)
4,208,255,20,30.244499,u04 (2200),u04 (2200)
...,...,...,...,...,...,...
29995,13166,255,9975,0.069678,nu (79),nu (79)
29996,13166,255,9980,0.070729,nu (79),nu (79)
29997,13166,255,9985,0.070542,nu (79),nu (79)
29998,13166,255,9990,0.069698,nu (79),nu (79)


In [111]:
# BOXPLOT FUNCTION
import matplotlib.pyplot as plt
plt.close('all')
def plot_box_plot(data, x_column, y_column, x_title, y_title, plot_title, ylim, box_colors):
    # print(data)
    p0 = ggplot(data, aes(x_column, y_column )) \
         + geom_jitter(alpha=0.3) \
         + geom_boxplot(alpha=0.7, notch=False, outlier_alpha=0, color = box_colors ) \
         + labs(x=x_title, y=y_title, title=plot_title) \
         + theme(axis_text_x=element_text(rotation=45, hjust=1)) \
         + coord_cartesian(ylim=ylim)              
    return p0


 # PLOT BOXPLOTS
plot_title = f'Maze {m[1:]} - Trace {t}'
lims = [0, 1]

box_plot_colors = ['black' for i in range(len(categories))]
box_plot_colors[position] = 'red'

save_name = f'Boxplots-{m}-T{int(t*10)}-l{lims[1]}.pdf'
print(f'      PLOT: BOXPLOT - {save_name}')
p = plot_box_plot(runtimes_last_episode, 'categories', 'steps', 'Layer', 'Extra Steps', plot_title, lims, box_plot_colors)
# ggsave(p, folder_nu + save_name, dpi=300, verbose = False)
p

      PLOT: BOXPLOT - Boxplots-M1-T0-l1.pdf


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ggplot: (134244205803)>

In [117]:
def plot_time_series(data, x_column, y_column, x_title, y_title, legend_title, plot_title, fill_cat, xlim, ylim):
    # print(data)
    p0 = ggplot(data, aes(x_column, y_column, color=fill_cat)) \
     + geom_line() \
     + labs(x=x_title, y=y_title, title=plot_title, caption='smt') \
     + coord_cartesian(xlim=xlim, ylim=ylim) \
     + scale_color_brewer(type="qual", palette=1, name=legend_title) 
    return p0


# plot runtimes:
plot_title = f'Maze {m[1:]} - Trace {t}'
ylims = [0, 1]
xlims = [0, 10000]

save_name = f'runtimes-{m}-T{int(t*10)}-y{ylims[1]}-x{xlims[1]}.pdf'
print(f'      PLOT: RUNTIME - {save_name}')
p = plot_time_series(erros_vs_episode_df, 'episode', 'steps' , 'Episode', 'Extra Steps', 'Group', plot_title , 'categories', xlims, ylims )
# ggsave(p, folder_nu_runtimes + save_name, dpi=300, verbose = False)
# erros_vs_episode_df
p

      PLOT: RUNTIME - runtimes-M1-T0-y1-x10000.pdf


c:\users\bucef\appdata\local\programs\python\python39\lib\site-packages\mizani\palettes.py:397: UserWarning: Warning message:Brewer palette Accent has a maximum of 8 colors Returning the palette you asked for with that many colors


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

c:\users\bucef\appdata\local\programs\python\python39\lib\site-packages\mizani\palettes.py:397: UserWarning: Warning message:Brewer palette Accent has a maximum of 8 colors Returning the palette you asked for with that many colors


<ggplot: (134244620285)>